## Automated trading for dual moving average crossover strategy on Single Asset using Interactive Brokers API

Reference: https://github.com/erdewit/ib_insync/blob/master/notebooks/

Remarks: haven't done yet

Prerequisites:
1. Install TWS API: https://interactivebrokers.github.io/#
2. `pip install ib_insync`

In [90]:
import yfinance as yf
from ib_insync import *

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)

In [91]:
asset_to_trade = "AAPL"
yahoo_stock_code = "AAPL"

market = "SMART"

currency = "USD"
cashQty = 1e4

In [92]:
ib = IB()
ib.connect("127.0.0.1", 7497, clientId=3) # replace with your connection details

<IB connected to 127.0.0.1:7497 clientId=3>

In [93]:
ib.positions()

[]

In [94]:
[v for v in ib.accountValues() if v.tag == 
 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU6799464', tag='NetLiquidationByCurrency', value='1022025.47', currency='BASE', modelCode='')]

In [95]:
# define the asset we're interested in
contract = Stock(asset_to_trade, market, currency)
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,CASHQTY,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,DUR,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,PEGMID,POSTATS,POSTONLY,PREOPGRTH,PRICECHK,REL,REL2MID,RELPCTOFS,RPI,RTH,SCALE,SCALEODD,SCALERST,SIZECHK,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,EDGX,FOXRIVER,PEARL,NYSENAT,LTSE,MEMX,IBEOS,OVERNIGHT,PSX', priceMagnifier=1, underConId=0, longName='APPLE INC', contractMonth='', industry='Technology', category='Computers', subcategory='Computers', t

In [96]:
# fetch data from yahoo finance
data= yf.download(yahoo_stock_code,start="2020-01-01",end="2022-12-31")


[*********************100%***********************]  1 of 1 completed


In [97]:
# Calculate moving average
data["MA7"] = data["Close"].rolling(window=7).mean()
data["MA14"] = data["Close"].rolling(window=14).mean()

In [98]:
# Create signals
data["Buy_Signal"] = data["MA7"] > data["MA14"]
data["Sell_Signal"] = data["MA7"] < data["MA14"]

In [99]:
#exchange_hours = ib.reqMarketDataType(contract)
#exchange_hours # cannot works and suspected need to buy the data

In [101]:
# check for buy signal
if data["Buy_Signal"].iloc[-1]:
    order = LimitOrder("BUY", 1, lmtPrice= 173.70, tif="IOC") # good-til-canceled
    trade = ib.placeOrder(contract, order)
    ib.sleep(1) # Wait a second for trade to execute
    print(trade)
    print("BUY")

# check for sell signal
elif data["Sell_Signal"].iloc[-1]:
    order = LimitOrder("SELL",1, lmtPrice = 177.60, tif="IOC") # good-til-canceled
    trade = ib.placeOrder(contract, order)
    print(trade)
    ib.sleep(1)
    print("SELL")
    

Trade(contract=Stock(symbol='AAPL', exchange='SMART', currency='USD'), order=LimitOrder(orderId=12, clientId=3, action='SELL', totalQuantity=1, lmtPrice=177.6, tif='IOC'), orderStatus=OrderStatus(orderId=12, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2023, 8, 14, 12, 32, 21, 438544, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')
SELL


Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm; ushmds; secdefil.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm; ushmds; secdefil.


In [ ]:
#ib.disconnect()